In [1]:
pip install Faker


Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
from faker import Faker
import random 

In [40]:
# Initialize Faker to generate synthetic data
fake = Faker()

# Set seed for reproducibility
Faker.seed(42)

# Generate synthetic data
data = []
for _ in range(1000):
    data.append({
        'Name': fake.name(),
        'Age': random.randint(18, 70),
        'Email': fake.email(),
        'Phone': fake.phone_number(),
        'Address': fake.address(),
        'Salary': random.randint(20000, 150000),
        'Join_Date': fake.date_this_decade(),
        'Employee_status': random.choice(['Full-Time', 'Part-Time', 'Contract']),
        'Department': random.choice(['IT', 'Engineering','Finance', 'HR', 'Marketing'])
    })
    


In [41]:
# Introduce missing values
for i in random.sample(range(len(data)), 50):
    data[i]['Email'] = None

# Introduce duplicate records
data.extend(random.sample(data, 100))
    
# Introduce outliers
for i in random.sample(range(len(data)), 20):
    data[i]['Salary'] = random.randint(200000, 500000)

In [42]:
# Create dataframe
df = pd.DataFrame(data)

In [43]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1100 entries, 0 to 1099
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Name             1100 non-null   object
 1   Age              1100 non-null   int64 
 2   Email            1046 non-null   object
 3   Phone            1100 non-null   object
 4   Address          1100 non-null   object
 5   Salary           1100 non-null   int64 
 6   Join_Date        1100 non-null   object
 7   Employee_status  1100 non-null   object
 8   Department       1100 non-null   object
dtypes: int64(2), object(7)
memory usage: 77.5+ KB


In [44]:
df.head()

,Name,Age,Email,Phone,Address,Salary,Join_Date,Employee_status,Department
0,Allison Hill,52,donaldgarcia@example.net,+1-219-560-0133,"79402 Peterson Drives Apt. 511\nDavisstad, PA ...",87043,2020-05-14,Contract,Engineering
1,Kimberly Dudley,59,smiller@example.net,+1-659-931-0341x316,"55341 Amanda Gardens Apt. 764\nLake Mark, WI 0...",148428,2021-12-12,Part-Time,Finance
2,Renee Morales,45,clarksherri@example.net,837-767-2423x88496,"710 Eric Estate\nCarlsonfurt, MS 78605",136897,2023-12-25,Contract,IT
3,Tricia Valencia,22,frazierdanny@example.net,001-645-514-6270x48281,"809 Burns Creek\nNatashaport, IA 08093",20089,2022-01-18,Full-Time,Marketing
4,Theresa Miller,41,wcabrera@example.net,001-822-778-2489x63834,"33150 Brianna Avenue Apt. 031\nPort Markhaven,...",22060,2024-02-17,Full-Time,Marketing


In [51]:
duplicates = df.duplicated().sum()
print("Number of duplicated rows:", duplicates)


Number of duplicated rows: 0


In [52]:
# Removing duplicate rows
df.drop_duplicates(inplace=True)


In [55]:
# Check for missing values
missing_values = df.isna().sum()
print("Number of missing values: ", missing_values)

Number of missing values:  Name                0
Age                 0
Email              50
Phone               0
Address             0
Salary              0
Join_Date           0
Employee_status     0
Department          0
dtype: int64


In [57]:
# Handling missing values by filling with a placeholder
df['Email'].fillna('unknown@example.com', inplace=True)

In [58]:
# Convert "Join_Date" to datetime
df['Join_Date'] = pd.to_datetime(df['Join_Date'])
print("Join_Date after conversion:")
print(df['Join_Date'].head())

Join_Date after conversion:
0   2020-05-14
1   2021-12-12
2   2023-12-25
3   2022-01-18
4   2024-02-17
Name: Join_Date, dtype: datetime64[ns]


In [76]:
# Creating a new feature 'Years_Employed' based on 'Join_Date'
df['Years_Employed'] = pd.Timestamp.now().year - df['Join_Date'].dt.year
df.head()

,Name,Age,Email,Phone,Address,Salary,Join_Date,Employee_status,Department,Years_Employed
0,Allison Hill,52,donaldgarcia@example.net,+1-219-560-0133,"79402 Peterson Drives Apt. 511\nDavisstad, PA ...",87043,2020-05-14,Contract,Engineering,4
1,Kimberly Dudley,59,smiller@example.net,+1-659-931-0341x316,"55341 Amanda Gardens Apt. 764\nLake Mark, WI 0...",148428,2021-12-12,Part-Time,Finance,3
2,Renee Morales,45,clarksherri@example.net,837-767-2423x88496,"710 Eric Estate\nCarlsonfurt, MS 78605",136897,2023-12-25,Contract,IT,1
3,Tricia Valencia,22,frazierdanny@example.net,001-645-514-6270x48281,"809 Burns Creek\nNatashaport, IA 08093",20089,2022-01-18,Full-Time,Marketing,2
4,Theresa Miller,41,wcabrera@example.net,001-822-778-2489x63834,"33150 Brianna Avenue Apt. 031\nPort Markhaven,...",22060,2024-02-17,Full-Time,Marketing,0


In [80]:
df['Address'] = df['Address'].str.replace('\n', ' ', regex = False)
df['Address'].head()

0    79402 Peterson Drives Apt. 511 Davisstad, PA 3...
1    55341 Amanda Gardens Apt. 764 Lake Mark, WI 07832
2                710 Eric Estate Carlsonfurt, MS 78605
3                809 Burns Creek Natashaport, IA 08093
4    33150 Brianna Avenue Apt. 031 Port Markhaven, ...
Name: Address, dtype: object

In [81]:
# Detecting outliers using z-score
z_scores = (df['Salary'] - df['Salary'].mean()) / df['Salary'].std()
outliers = df[abs(z_scores) > 3]
print("Outliers based on Salary:")
print(outliers[['Name', 'Salary']].head())


Outliers based on Salary:
              Name  Salary
212       Mary May  490597
318   Donna Hardin  387636
409  Michael Smith  403764
606    Julie Smith  485256
640   Joshua Perry  448590


In [83]:
df_cleaned = df.drop(outliers.index)

In [84]:
df_cleaned

,Name,Age,Email,Phone,Address,Salary,Join_Date,Employee_status,Department,Years_Employed
0,Allison Hill,52,donaldgarcia@example.net,+1-219-560-0133,"79402 Peterson Drives Apt. 511 Davisstad, PA 3...",87043,2020-05-14,Contract,Engineering,4
1,Kimberly Dudley,59,smiller@example.net,+1-659-931-0341x316,"55341 Amanda Gardens Apt. 764 Lake Mark, WI 07832",148428,2021-12-12,Part-Time,Finance,3
2,Renee Morales,45,clarksherri@example.net,837-767-2423x88496,"710 Eric Estate Carlsonfurt, MS 78605",136897,2023-12-25,Contract,IT,1
3,Tricia Valencia,22,frazierdanny@example.net,001-645-514-6270x48281,"809 Burns Creek Natashaport, IA 08093",20089,2022-01-18,Full-Time,Marketing,2
4,Theresa Miller,41,wcabrera@example.net,001-822-778-2489x63834,"33150 Brianna Avenue Apt. 031 Port Markhaven, ...",22060,2024-02-17,Full-Time,Marketing,0
...,...,...,...,...,...,...,...,...,...,...
995,Bobby Brown,56,michaelvalencia@example.net,552.311.2759x637,"10886 Terri Center Dennismouth, WY 76714",66122,2022-03-16,Part-Time,Marketing,2
996,Allison Young,21,hicksandrew@example.net,(248)351-0021x82414,"461 Walters Mountains Mayborough, HI 08919",137248,2022-05-24,Contract,IT,2
997,Laura Atkins,53,clarkdavid@example.org,834-774-3344x210,"46010 Rodriguez Crossing West Michael, WA 60166",54997,2022-07-31,Part-Time,Engineering,2
998,Nathan Wolf,37,gabrielleanderson@example.org,+1-956-972-1761x898,"751 Vincent Mountain Apt. 239 New Tashabury, R...",29189,2021-05-14,Part-Time,Finance,3
